In [ ]:
import os
import io
import contextlib
from dotenv import load_dotenv

load_dotenv()

In [ ]:
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")
assert GROQ_API_KEY is not None

In [ ]:
BRIGHT_DATA_SERP_API_KEY=os.environ.get("BRIGHT_DATA_SERP_API_KEY")
assert BRIGHT_DATA_SERP_API_KEY is not None

In [ ]:
from langchain_brightdata import BrightDataSERP

serp_tool=BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_SERP_API_KEY,
    search_engine="google",
    country="us",
    parse_results=True

)

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [ ]:
def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [ ]:
from langgraph.prebuilt import create_react_agent


weather_agent = create_react_agent(
    model=model,  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
results = weather_agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in philly"}]}
)

last_message = results["messages"][-1].content
print(last_message)


In [ ]:
from langgraph.prebuilt import create_react_agent


search_agent = create_react_agent(
    model=model,  
    tools=[serp_tool],  
    prompt="You are a helpful assistant that can search the internet"  
)

# Run the agent
results = search_agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in philly"}]}
)

results["messages"][-1].content




In [ ]:
for chunk in search_agent.stream(
    {"messages":[{"role":"user","content":"what is the weather in philly"}]}

    
):
    agent_result=chunk.get('agent')
    if agent_result:
        print(agent_result)
        agent_msgs=agent_result.get("messages")
        if not agent_msgs:
            continue 
        txt_response=agent_msgs[-1].content
        if txt_response:
            print(txt_response)